In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# urlの取得
def url_to_soup(url):
    req = requests.get(url)
    return BeautifulSoup(req.content, 'html.parser')

In [3]:
def result_data(url):
    soup = url_to_soup(url)

    # 土の状態
    condition = soup.find(id="race-data02").get_text().replace('\n','').split(';')[1].split('　')[2][0:1]

    # レースの長さ
    race_len = int(soup.find(id="race-data01-a").get_text().replace('\n','').split('　')[3].replace(',','')[1:5])

    # レース日
    race_date = soup.find(id="race-data01-a").get_text().replace('\n','').split(';')[0].split('第')[0]
    return condition, race_len, race_date

In [4]:
re_da = list(result_data("https://www.nankankeiba.com/race_info/2019120620140512.do"))
re_da[1]

1400

In [68]:
# csvファイルの読み込み
# 欠損地を含む行を削除
# indexの振り直し
a = pd.read_csv("3.csv", index_col=0).dropna(how='any', axis=0)
a = a.reset_index(drop=True)
print(len(a))
# a = a[0:10].reset_index(drop=True)

25


In [69]:
type(a.loc[1, '距離'])

str

In [70]:
# 競馬場の判定
for i in range(len(a)):
    if "大井" in a.loc[i, "競馬場"]:
        a.loc[i, '競馬場'] = 1
    else:
        a.loc[i, '競馬場'] = 0

# 大井競馬場であるレースだけを抽出
a = a[a['競馬場'] == 1].drop('競馬場', axis=1)
a = a.reset_index(drop=True)
#問題解決URL
# http://hirotaka-hachiya.hatenablog.com/entry/2016/01/04/171349
# 列を特定してから行を特定しているため二段階になり時間がかかってしまう

In [71]:
a.loc[1, '距離']

'1200'

In [72]:
# 距離の判定
for i in range(len(a)):
    if a.loc[i,'距離'] == re_da[1]:
        a.loc[i, '距離'] = 0
    else:
        a.loc[i, '距離'] = int(a.loc[i, '距離']) - 1400

In [73]:
# 着順変換
for i in range(len(a)):
    a.loc[i, '着順'] = a.loc[i, '着順'].split("/")[0]

In [74]:
# タイムを秒数に変換
from datetime import datetime as dt
for i in range(len(a)):
    time = dt.strptime(a.loc[i, 'タイム'] , '%M:%S.%f') - dt(1900, 1,1)
    time = time.total_seconds()
    a.loc[i, 'タイム'] = str(time)
#     a.loc[i, 'タイム'] = time.strftime('%M:%S.%f')

In [75]:
a

,距離,天候馬場,人気,着順,タイム,体重,負担重量
0,0,晴/良,10,4,88.9,454,56.0
1,-200,曇/重,15,9,74.3,453,55.0
2,0,曇/良,11,11,89.8,455,56.0
3,0,雨/不良,12,10,89.8,455,56.0
4,0,晴/良,10,3,87.5,448,56.0
5,0,晴/重,12,5,88.3,447,56.0
6,-200,曇/重,9,7,75.1,447,56.0
7,-200,曇/稍重,15,15,75.4,445,56.0
8,0,曇/重,10,11,90.5,444,56.0
9,-200,晴/良,10,4,73.8,452,56.0


In [76]:
# 天候馬場のデータを整形
for i in range(len(a)):
    if '/重' == a.loc[i, '天候馬場'][-2:]:
        a.loc[i, 'soil_heavy'] = 1
    else:
        a.loc[i, 'soil_heavy'] = 0
for i in range(len(a)):
    if '稍重' == a.loc[i, '天候馬場'][-2:]:
        a.loc[i, 'soil_s_heavy'] = 1
    else:
        a.loc[i, 'soil_s_heavy'] = 0
for i in range(len(a)):
    if '/良' == a.loc[i, '天候馬場'][-2:]:
        a.loc[i, 'soil_good'] = 1
    else:
        a.loc[i, 'soil_good'] = 0 
for i in range(len(a)):
    if '不良' == a.loc[i, '天候馬場'][-2:]:
        a.loc[i, 'soil_bad'] = 1
    else:
        a.loc[i, 'soil_bad'] = 0 


# 天候馬場の列を削除
a = a.drop('天候馬場', axis=1)


In [17]:
a.to_csv('a' + str(3) + '.csv')

In [77]:
a

,距離,人気,着順,タイム,体重,負担重量,soil_heavy,soil_s_heavy,soil_good,soil_bad
0,0,10,4,88.9,454,56.0,0.0,0.0,1.0,0.0
1,-200,15,9,74.3,453,55.0,1.0,0.0,0.0,0.0
2,0,11,11,89.8,455,56.0,0.0,0.0,1.0,0.0
3,0,12,10,89.8,455,56.0,0.0,0.0,0.0,1.0
4,0,10,3,87.5,448,56.0,0.0,0.0,1.0,0.0
5,0,12,5,88.3,447,56.0,1.0,0.0,0.0,0.0
6,-200,9,7,75.1,447,56.0,1.0,0.0,0.0,0.0
7,-200,15,15,75.4,445,56.0,0.0,1.0,0.0,0.0
8,0,10,11,90.5,444,56.0,1.0,0.0,0.0,0.0
9,-200,10,4,73.8,452,56.0,0.0,0.0,1.0,0.0
